In [1]:
import pandas as pd

In [2]:
cd /content/drive/MyDrive/ARPIX/ReliableQA/build_dataset

/content/drive/MyDrive/ARPIX/ReliableQA/build_dataset


In [3]:
df = pd.read_csv('dataset+.csv')

In [4]:
cd /content/drive/MyDrive/ARPIX/ReliableQA/build_classifier

/content/drive/MyDrive/ARPIX/ReliableQA/build_classifier


In [6]:
pip install sentencepiece

     |████████████████████████████████| 1.2 MB 3.8 MB/s 


In [ ]:
pip install transformers

In [9]:
import pandas as pd
import numpy as np
import os
import gc

import random
import sentencepiece

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# set a seed value
torch.manual_seed(555)

from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score, accuracy_score

import transformers
from transformers import BertTokenizer, BertForSequenceClassification 
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification
from transformers import AdamW

import warnings
warnings.filterwarnings("ignore")


print(torch.__version__)

1.10.0+cu111


In [10]:
df = shuffle(df)

In [13]:
df = df.reset_index(drop=True)

In [18]:
df.head()

,Unnamed: 0,class,fine_class,text,target
0,18593,CNT,E2E,Why was Maisie afraid that she would be expect...,CNT
1,1238,TRS,K2E,오늘 밤이 영어로 뭐에요?,오늘 밤
2,6311,STC,E2E,Can you use good in a sentence?,good
3,15516,SPL,M2E,use의 철자는 무엇인가?,use
4,19726,CNT,E2E,Who received the gift of beauty?,CNT


In [19]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
result = le.fit_transform(df['class'])
print(result)
# array([1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1,
#        0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0,
#        0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1,
#        0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
#        0, 1, 1, 1, 1.... 1, 1])

# 변환된 label classes 확인
le.classes_
# array(['female', 'male'], dtype=object)

[0 5 4 ... 0 1 4]


array(['CNT', 'PRN', 'RPT', 'SPL', 'STC', 'TRS'], dtype=object)

In [21]:
df['label']=pd.Series(result)

In [23]:
df.head(20)

,Unnamed: 0,class,fine_class,text,target,label
0,18593,CNT,E2E,Why was Maisie afraid that she would be expect...,CNT,0
1,1238,TRS,K2E,오늘 밤이 영어로 뭐에요?,오늘 밤,5
2,6311,STC,E2E,Can you use good in a sentence?,good,4
3,15516,SPL,M2E,use의 철자는 무엇인가?,use,3
4,19726,CNT,E2E,Who received the gift of beauty?,CNT,0
5,14165,PRN,M2E,word는 어떻게 말하는가.,word,1
6,11316,PRN,E2E,Can you pronounce the ease?,ease,1
7,14019,PRN,M2E,answer는 어떻게 말하죠?,answer,1
8,1322,TRS,K2E,마을은 영어로 뭐야?,마을,5
9,16103,SPL,M2E,story는 어떻게 써요?,story,3


동일한 data distribution을 갖도록 train, val, test set으로 split하기

In [68]:
def get_class_counts(df):
    grp = df.groupby(['class'])['Unnamed: 0'].nunique()
    return ({key: grp[key] for key in list(grp.keys())})

In [70]:
def get_class_proportions(df):
    class_counts = get_class_counts(df)
    return {val[0]: round(val[1]/df.shape[0], 4) for val in class_counts.items()}

In [72]:
print(get_class_counts(df))
get_class_proportions(df)

{'CNT': 4000, 'PRN': 6264, 'RPT': 32, 'SPL': 2265, 'STC': 2717, 'TRS': 5480}


{'CNT': 0.1927,
 'PRN': 0.3018,
 'RPT': 0.0015,
 'SPL': 0.1091,
 'STC': 0.1309,
 'TRS': 0.264}

In [77]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2, stratify=df['class'])

train_class_proportions = get_class_proportions(train)
test_class_proportions = get_class_proportions(test)

print(train_class_proportions)
print(test_class_proportions)

{'CNT': 0.1927, 'PRN': 0.3018, 'RPT': 0.0016, 'SPL': 0.1091, 'STC': 0.1309, 'TRS': 0.264}
{'CNT': 0.1927, 'PRN': 0.3018, 'RPT': 0.0014, 'SPL': 0.1091, 'STC': 0.131, 'TRS': 0.264}


In [79]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(train, test_size=0.25, stratify=train['class'])

train_class_proportions = get_class_proportions(train)
val_class_proportions = get_class_proportions(val)

print(train_class_proportions)
print(val_class_proportions)

{'CNT': 0.1927, 'PRN': 0.3018, 'RPT': 0.0015, 'SPL': 0.1091, 'STC': 0.1309, 'TRS': 0.264}
{'CNT': 0.1927, 'PRN': 0.3018, 'RPT': 0.0017, 'SPL': 0.1091, 'STC': 0.1308, 'TRS': 0.264}


In [80]:
print(len(train))
print(len(val))
print(len(test))

12454
4152
4152


In [61]:
# 모델 input으로 적절한 길이는 어느정도인가?

count = df['text'].str.split().apply(len)
count.describe()

count    20758.000000
mean         5.622459
std          2.953268
min          1.000000
25%          3.000000
50%          5.000000
75%          7.000000
max         25.000000
Name: text, dtype: float64

In [25]:
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification

MODEL_TYPE = 'xlm-roberta-base'
print('Loading XLMRoberta tokenizer...')
tokenizer = XLMRobertaTokenizer.from_pretrained(MODEL_TYPE)

Loading XLMRoberta tokenizer...


Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

In [33]:
L_RATE = 1e-5
MAX_LEN = 20
NUM_EPOCHS = 3
BATCH_SIZE = 16
NUM_CORES = os.cpu_count()

NUM_CORES

2

In [36]:
class CompDataset(Dataset):

    def __init__(self, df):
        self.df_data = df



    def __getitem__(self, index):

        # get the sentence from the dataframe
        sentence1 = self.df_data.loc[index, 'text']
        # sentence2 = self.df_data.loc[index, 'hypothesis']

        # Process the sentence
        # ---------------------

        encoded_dict = tokenizer.encode_plus(
                    sentence1,           # Sentences to encode.
                    add_special_tokens = True,      # Add the special tokens.
                    max_length = MAX_LEN,           # Pad & truncate all sentences.
                    pad_to_max_length = True,
                    return_attention_mask = True,   # Construct attn. masks.
                    return_tensors = 'pt',          # Return pytorch tensors.
               )
        
        # These are torch tensors.
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        
        # Convert the target to a torch tensor
        target = torch.tensor(self.df_data.loc[index, 'label'])

        sample = (padded_token_list, att_mask, target)


        return sample


    def __len__(self):
        return len(self.df_data)
    
    
    
    
    

class TestDataset(Dataset):

    def __init__(self, df):
        self.df_data = df



    def __getitem__(self, index):

        # get the sentence from the dataframe
        sentence1 = self.df_data.loc[index, 'text']
        # sentence2 = self.df_data.loc[index, 'hypothesis']

        # Process the sentence
        # ---------------------

        encoded_dict = tokenizer.encode_plus(
                    sentence1,           # Sentence to encode.
                    add_special_tokens = True,      # Add the special tokens.
                    max_length = MAX_LEN,           # Pad & truncate all sentences.
                    pad_to_max_length = True,
                    return_attention_mask = True,   # Construct attn. masks.
                    return_tensors = 'pt',          # Return pytorch tensors.
               )
        
        # These are torch tensors.
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        
               

        sample = (padded_token_list, att_mask)


        return sample


    def __len__(self):
        return len(self.df_data)

In [83]:
train_data = CompDataset(train)
val_data = CompDataset(val)
test_data = TestDataset(test)

train_dataloader = torch.utils.data.DataLoader(train_data,
                                        batch_size=BATCH_SIZE,
                                        shuffle=True,
                                       num_workers=NUM_CORES)

val_dataloader = torch.utils.data.DataLoader(val_data,
                                        batch_size=BATCH_SIZE,
                                        shuffle=True,
                                       num_workers=NUM_CORES)

test_dataloader = torch.utils.data.DataLoader(test_data,
                                        batch_size=BATCH_SIZE,
                                        shuffle=False,
                                       num_workers=NUM_CORES)



print(len(train_dataloader))
print(len(val_dataloader))
print(len(test_dataloader))

779
260
260


In [50]:
# Get one train batch

padded_token_list, att_mask, target = next(iter(train_dataloader))

print(padded_token_list.shape)
print(att_mask.shape)
print(target.shape)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


torch.Size([16, 20])
torch.Size([16, 20])
torch.Size([16])


In [51]:
example = padded_token_list[0]

In [52]:
a = tokenizer.decode(example,
                skip_special_tokens=False)

b = tokenizer.decode(example,
                skip_special_tokens=True)

print(a)
print(b)

<s> Why was Maisie afraid that she would be expected to spin a great deal at the Castle</s>
Why was Maisie afraid that she would be expected to spin a great deal at the Castle


In [ ]:
from transformers import XLMRobertaForSequenceClassification

model = XLMRobertaForSequenceClassification.from_pretrained(
    MODEL_TYPE, 
    num_labels = 3, # The number of output labels. 2 for binary classification.
)

# Send the model to the device.
model.to(device)

In [12]:
cd /content/drive/MyDrive/ARPIX/ReliableQA/build_classifier

/content/drive/MyDrive/ARPIX/ReliableQA/build_classifier


In [13]:
from data_loader import *

In [14]:
df = label_shuffle_data()

['CNT' 'PRN' 'RPT' 'SPL' 'STC' 'TRS']


In [15]:
df.loc[:100]

,Unnamed: 0,class,fine_class,text,target,label
2579,2579,TRS,M2E,have는 무슨 뜻이에요?,have,5
12937,12937,PRN,E2E,Where's the off stress?,off,1
1638,1638,TRS,M2E,water의 의미가 뭐에요?,water,5
1849,1849,TRS,M2E,start의 의미는 무엇인가?,start,5
13966,13966,PRN,M2E,off는 어떻게 말하죠?,off,1
...,...,...,...,...,...,...
6246,6246,STC,E2E,Can you use open in a sentence?,open,4
17342,17342,CNT,E2E,What did Fionn find when he woke up?,CNT,0
10735,10735,PRN,E2E,How do you say a eye in English?,eye,1
18984,18984,CNT,E2E,What did the swallow do a lot of this year?,CNT,0


In [1]:
train, val, test = train_val_test_split(df, 0.2, 0.2)

NameError: ignored